 
1.   Write some markdown to explain that this notebook will be mainly used for the capstone project.
2.   Import the pandas library as pd.
3.   Import the Numpy library as np.
4.   Print the following the statement: Hello Capstone Project Course!

In [1]:
import pandas as pd
import numpy as np

statement = 'Hello Capstone Project Course!'
print(statement)

Hello Capstone Project Course!
